## Cruising through Soton: Template code for Google Routes API (16th Jan 2024)

Set up your free trial google developer trial (requires a card for billing account but will not charge you anything).

Config is a hidden file which stores your api key for google maps. 

Example HTTP request below. The time is set for very earlier morning when traffic conditions are normally good.

In [ ]:
import config
import requests

api_key = config.api_key

origin = 'YOUR, HOME, ADDRESS, HERE'
destination = 'YOUR, WORK, ADDRESS, HERE'

url = "https://routes.googleapis.com/directions/v2:computeRoutes"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    "X-Goog-FieldMask": "routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline"}

data = {
    "origin": {"address": origin},
    "destination": {"address": destination},
    "travelMode": "DRIVE",
    "trafficModel": "BEST_GUESS", # PESSIMISTIC, OPTIMISTIC, BEST_GUESS
    "routingPreference": "TRAFFIC_AWARE_OPTIMAL", 
    "departureTime": "2030-02-05T02:00:00Z",
    "computeAlternativeRoutes": False,
    "routeModifiers": {"avoidTolls": False,"avoidHighways": False,"avoidFerries": False},
    "languageCode": "en-UK",
    "units": "METRIC"}

response = requests.post(url, json=data, headers=headers)
quickest_journey = response.json()
print(quickest_journey)

To loop over many departure times and different traffic conditions, use this additional cell below. Add a way point along the way to ensure the route will not change to a different route depending on traffic conditions.


In [ ]:
import numpy as np
import datetime

waypoint = 'A PLACE ALONG THE WAY' # optional if you want to guarantee the route goes the same way every time. 
# If you don't specify a waypoint, the route will be calculated based on traffic conditions at the time of the request.

driving_guesses = ['PESSIMISTIC', 'OPTIMISTIC', 'BEST_GUESS'] # Traffic conditions

day = 9
month = 2
year = 2030
earliest_leaving_time = datetime.datetime(year, month, day, 15, 0, 0) # starting time
date_times = [earliest_leaving_time + datetime.timedelta(minutes=i) for i in range(0, 241, 2)] # 0 to 240 minutes in 2 minute intervals

# covert all date_times to example_formats
times = [date_time.strftime("%Y-%m-%dT%H:%M:%SZ") for date_time in date_times]
print(times) # double check times are correct

duration_dict = {} # empty dictionary to store the durations for each traffic condition

url = "https://routes.googleapis.com/directions/v2:computeRoutes"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    "X-Goog-FieldMask": "routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline"}

for guess in driving_guesses: # traffic conditions
    duration_times = np.array([]) # empty array to store the durations for each time
    for time in times:
        data = {
            "origin": {"address": origin},
            "destination": {"address": destination},
            "intermediates": [{"address": waypoint, "via": True}],
            "travelMode": "DRIVE",
            "trafficModel": guess, # PESSIMISTIC, OPTIMISTIC, BEST_GUESS
            "routingPreference": "TRAFFIC_AWARE_OPTIMAL", # TRAFFIC_AWARE_OPTIMAL, TRAFFIC_UNAWARE (remove departure time)
            "departureTime": time,
            "computeAlternativeRoutes": False,
            "routeModifiers": {"avoidTolls": False, "avoidHighways": False, "avoidFerries": False},
            "languageCode": "en-UK",
            "units": "METRIC"}

        response = requests.post(url, json=data, headers=headers)
        quickest_journey = response.json()
        # removing the s from the string to get the seconds integer. 
        duration_times = np.append(duration_times, int(quickest_journey['routes'][0]['duration'][:-1]))
    duration_dict[guess] = duration_times

 An example code block for plotting your journey times is below

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
plt.style.use('Solarize_Light2')

timeformat = DateFormatter('%H:%M')
for guess in driving_guesses:
    plt.plot(date_times, duration_dict[guess]/60, label=guess) # /60 converts seconds to minutes
plt.axhline(int(quickest_journey['routes'][0]['duration'][:-1])/60, color='r', linestyle='--', label='Quickest "Legal" Journey')
plt.gca().xaxis.set_major_formatter(timeformat)
plt.xlabel("Departure Time")
plt.ylabel("Journey Duration (mins)")
plt.title(f" The Journey Time on {date_times[0].strftime('%a %d/%m/%y')}")
plt.legend()
plt.show()


Extra code I used to generate the plots for the talk, including your ETA as a function of departure time. The Extra commuting time and emotional salary drop as a function of departure time. Also includes the research plot of commute dissatification as a function of time commuting. 

In [ ]:
etas = {}
for guess in driving_guesses:
    etas[guess] = np.array([])
    for index, time in enumerate(duration_dict[guess]):
        #print(index, time)
        eta = date_times[index] + datetime.timedelta(minutes=time/60)
        etas[guess] = np.append(etas[guess], eta)


for index, guess in enumerate(driving_guesses):
    plt.plot(date_times, etas[guess], label=guess)

#plt.axhline(datetime.datetime(2024, 2, day, 8, 10, 0), color='r', linestyle='--', label="Late ⬆")
plt.gca().xaxis.set_major_formatter(timeformat)
plt.gca().yaxis.set_major_formatter(timeformat)
plt.legend()
plt.xlabel("Departure Time")
plt.ylabel("Arrival Time")
plt.title(f"Austen's Commute to Uni Through Hedge End \n Estimated Arrival Time on {date_times[0].strftime('%a %d/%m/%y')}")
plt.show()


# print the time when the arrival time is greater than 8:10
for guess in driving_guesses:
    for index, time in enumerate(etas[guess]):
        if time > datetime.datetime(year, month, day, 8, 10, 0):
            print(f"{guess} - {date_times[index]}")
            break

for index, guess in enumerate(driving_guesses):
    plt.plot(date_times, etas[guess], label=guess)

plt.gca().xaxis.set_major_formatter(timeformat)
plt.gca().yaxis.set_major_formatter(timeformat)
plt.legend()
plt.xlabel("Departure Time")
plt.ylabel("Arrival Time")
plt.ylim(datetime.datetime(year, month, day, 7, 30, 0), datetime.datetime(year, month, day, 8, 30, 0))
plt.xlim(datetime.datetime(year, month, day, 6, 30, 0), datetime.datetime(year, month, day, 8, 0, 0))
plt.title(f"Austen's Commute to Uni Through Hedge End \n Estimated Arrival Time on {date_times[0].strftime('%a %d/%m/%y')}")
plt.show()

# Extra Commute Time and Emotional Salary Drop \n for Different Departure Times
deltas = {}
for guess in driving_guesses:
    deltas[guess] = np.array([])
    for time in duration_dict[guess]:
        delta = (time - int(quickest_journey['routes'][0]['duration'][:-1]))/60
        deltas[guess] = np.append(deltas[guess], delta)

# Research data points
increased_commute = [0, 10]
salary_percent_drop = [0, -19]
predicted_commute = np.arange(0, 90, 1)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(np.array(increased_commute).reshape(-1, 1), np.array(salary_percent_drop).reshape(-1, 1))
predicted_salary_percent_drop = model.predict(np.array(predicted_commute).reshape(-1, 1))
# gradient coefficient
print(model.coef_)

plt.plot(predicted_commute, predicted_salary_percent_drop)
plt.scatter(increased_commute, salary_percent_drop, s=100, c='r', marker='x', label="Data")
plt.xlabel("Extra Commute Time(mins)")
plt.ylabel("Emotional Salary Drop (%)")
plt.ylim(-100, 0)
plt.xlim(0, 90)
plt.title("Relationship Between Extra Commute Time and \n Emotional Salary Drop")
plt.legend()
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for guess in driving_guesses:
    ax1.plot(date_times, deltas[guess], label=guess)
    salary_delta = deltas[guess] * -model.coef_
    ax2.plot(date_times, salary_delta[0], label=guess)

ax2.set_ylabel("Emotional Salary Drop (%)")
ax1.set_xlabel("Departure Time")
ax1.set_ylabel("Extra Commute Time (mins)")
plt.gca().xaxis.set_major_formatter(timeformat)
plt.title(f"Extra Commute Time and Emotional Salary Drop \n for Different Departure Times on {date_times[0].strftime('%a %d/%m/%y')}")
plt.legend()
plt.show()

# Route satisfaction level. 
#import csv data
traffic_higgins_data = np.genfromtxt('Traffic Higgins et al. 2017.csv', delimiter=',')
clear_higgins_data = np.genfromtxt('Clear Higgins et al. 2017.csv', delimiter=',')
plt.plot(traffic_higgins_data[:,0], traffic_higgins_data[:,1], label="3-4 Days of Traffic Jams")
plt.plot(clear_higgins_data[:,0], clear_higgins_data[:,1], label="3-4 Days of Clear Roads")
plt.xlabel("One-way Travel Time (mins)")
plt.ylabel("Commute Dissatisfaction (%)")
plt.title("Relationship Between Commute Dissatisfaction and  \n Travel Time - Higgins et al. 2017")
plt.legend()
plt.ylim(0, 100)
plt.xlim(0,90)
plt.show()
